In [1]:
from pyspark import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import Bucketizer
from pyspark.ml.stat import Correlation
from pyspark.ml.linalg import Vectors
from pyspark.mllib.stat import Statistics
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

#### Exploratory data analysis (EDA) - Part 1

In [3]:
# Carrega dataset do desafio (basta descompactar todos na mesma pasta)
#display(dbutils.fs.ls("dbfs:/FileStore/tables/"))
df_desafio = spark.read.format("csv").options(header='true').load("/FileStore/tables/*.csv")

In [4]:
# Elimina na's e duplicados do df
df_desafio_v2 = df_desafio.dropna(how='any').dropDuplicates()

# Ajusta tipo de colunas
df_desafio_v2 = df_desafio_v2.selectExpr(
  'cast(time as timestamp) time',
  'ca',
  'unit',
  'scp',
  'station',
  'linename',
  'division',
  'desc',
  'cast(entries as int) entries',
  'cast(exits as int) exits'
)

# Features para visão temporal
df_desafio_v2 = df_desafio_v2.withColumn(
  "dt_year",
  year(col("time"))
).withColumn(
  "dt_month",
  month(col("time"))
).withColumn(
  "dt_day",
  dayofmonth(col("time"))
).withColumn(
  "dt_dayofy",
  dayofyear(col("time"))  
).withColumn(
  "dt_hour",
  hour(col("time"))
).withColumn(
  "dt_min",
  minute(col("time"))
).withColumn(
  "dt_week_no",
  weekofyear(col("time"))
).withColumn(
  "dt_int",
  unix_timestamp(col("time"))
).withColumn(
  "dt_month_year",
  date_format(col("time"), "Y-MM")
)

In [5]:
# Intervalos para buckets
splits = [-30000000, -20000000, -10000000, 0.0, 10000000, 20000000, 30000000]

# ===> Bucket: 'entries'
# dataFrame = df_desafio_v2.select(col('entries'))
# bucketizer = Bucketizer(splits=splits, inputCol="entries", outputCol="bucketedFeatures")
# bucketedData_entries = bucketizer.transform(dataFrame)
# sorted(bucketedData_entries.groupBy("bucketedFeatures").count().collect())
# [Row(bucketedFeatures=0.0, count=15721),
#  Row(bucketedFeatures=1.0, count=4028),
#  Row(bucketedFeatures=2.0, count=496),
#  Row(bucketedFeatures=4.0, count=71457425), ==> Begin
#  Row(bucketedFeatures=5.0, count=5052549),  <== End
#  Row(bucketedFeatures=6.0, count=278162),
#  Row(bucketedFeatures=7.0, count=2237294)]

# ===> Bucket: 'exits'
# dataFrame = df_desafio_v2.select(col('exits'))
# bucketizer = Bucketizer(splits=splits, inputCol="exits", outputCol="bucketedFeatures")
# bucketedData_exits = bucketizer.transform(dataFrame)
# sorted(bucketedData_exits.groupBy("bucketedFeatures").count().collect())
# [Row(bucketedFeatures=0.0, count=30576),
#  Row(bucketedFeatures=2.0, count=576),
#  Row(bucketedFeatures=3.0, count=1),
#  Row(bucketedFeatures=4.0, count=73751241), ==> Begin
#  Row(bucketedFeatures=5.0, count=3698911),  <== End
#  Row(bucketedFeatures=6.0, count=413740),
#  Row(bucketedFeatures=7.0, count=1150630)]

# Parâmetros para filtros de outliers
outlier_begin = 0
outlier_end = 20000000

df_desafio_v2 = df_desafio_v2.where((col('entries')>=outlier_begin) & (col('entries')<=outlier_end) & (col('exits')>=outlier_begin) & (col('exits')<=outlier_end))

In [6]:
# Check: 79.609.191 / 79.130.015 / 79.045.675 / 75.923.980
count_desafio = df_desafio.count()
count_desafio_na = df_desafio.dropna(how='any').count()
count_desafio_final = df_desafio.dropna(how='any').dropDuplicates().count()
count_desafio_outliers = df_desafio_v2.count()

df_amostras = sc.parallelize([
  ('antes',count_desafio,0,0,0,0),
  ('depois',0,count_desafio_final,count_desafio-count_desafio_na,count_desafio_na-count_desafio_final,count_desafio_final-count_desafio_outliers)
]).toDF(['AMOSTRAS','TOTAL','UNICO','NA','DUPLICADO','OUTLIERS'])

display(df_amostras)

In [7]:
df_amostras_v2 = sc.parallelize([
  ('',count_desafio-count_desafio_na,count_desafio_na-count_desafio_final,count_desafio_final-count_desafio_outliers)
]).toDF(['AMOSTRAS','NA','DUPLICADO','OUTLIERS'])

display(df_amostras_v2)

In [8]:
# Método para variáveis categóricas (dummys) ~14.59 minutes
lista_idx = ['ca', 'unit', 'scp', 'station', 'linename', 'division', 'desc']
indexers = [StringIndexer(inputCol=column,outputCol=column+"_idx").fit(df_desafio_v2) for column in lista_idx]
pipeline = Pipeline(stages=indexers)
df_desafio_v2 = pipeline.fit(df_desafio_v2).transform(df_desafio_v2)
#display(df_desafio_v2)

In [9]:
df_desafio_v2.write.parquet("/FileStore/tables/df_desafio.parquet")
df_desafio_num.write.parquet("/FileStore/tables/df_desafio_ml.parquet")

#### Exploratory data analysis (EDA) - Part 2

In [11]:
# Carrega Parquet
df_desafio_pqt = spark.read.parquet("/FileStore/tables/df_desafio.parquet")
df_desafio_num = df_desafio_pqt.select(
  'entries','exits','dt_year','dt_month','dt_day','dt_dayofy',
  'dt_hour','dt_min','dt_week_no','dt_int','ca_idx','unit_idx',
  'scp_idx','station_idx','linename_idx','division_idx','desc_idx'
)

In [12]:
# col_names = df_desafio.columns
# features = df_desafio.rdd.map(lambda row: row[0:])
# corr_mat = Statistics.corr(features, method="pearson")
# corr_df = pd.DataFrame(corr_mat)
# corr_df.index, corr_df.columns = col_names, col_names
# spark.createDataFrame(corr_df).write.parquet("/FileStore/tables/corr_df.parquet")

Out[ 4 ]: 
 entries exits dt_year dt_month dt_day dt_dayofy \
entries 1.000000 0.620563 0.055824 -0.001030 0.001007 -0.000892 
exits 0.620563 1.000000 0.037681 -0.002759 0.000771 -0.002649 
dt_year 0.055824 0.037681 1.000000 -0.163327 -0.010380 -0.163039 
dt_month -0.001030 -0.002759 -0.163327 1.000000 0.002659 0.996308 
dt_day 0.001007 0.000771 -0.010380 0.002659 1.000000 0.088203 
dt_dayofy -0.000892 -0.002649 -0.163039 0.996308 0.088203 1.000000 
dt_hour -0.010027 -0.009431 -0.016023 -0.001062 -0.000554 -0.001124 
dt_min -0.129828 -0.110890 -0.073464 -0.005232 0.000708 -0.005282 
dt_week_no -0.000623 -0.002484 -0.154780 0.968979 0.060967 0.970475 
dt_int 0.056426 0.037813 0.991417 -0.033431 0.001171 -0.032651 
ca_idx 0.033550 0.030149 0.026807 -0.001028 0.001031 -0.000929 
unit_idx 0.048877 0.019794 0.022612 -0.001818 0.000714 -0.001745 
scp_idx -0.127529 -0.128658 0.016085 -0.000523 -0.000172 -0.000546 
station_idx -0.001799 -0.019425 0.224509 -0.027903 0.000933 -0.027375 
linename_idx 0.076172 0.084422 0.054080 -0.003617 -0.001351 -0.003726 
division_idx -0.162791 -0.128089 0.009763 0.002106 -0.000357 0.002045 
desc_idx -0.018528 -0.012205 -0.213068 -0.002147 0.007103 -0.001850 

 dt_hour dt_min dt_week_no dt_int ca_idx unit_idx \
entries -0.010027 -0.129828 -0.000623 0.056426 0.033550 0.048877 
exits -0.009431 -0.110890 -0.002484 0.037813 0.030149 0.019794 
dt_year -0.016023 -0.073464 -0.154780 0.991417 0.026807 0.022612 
dt_month -0.001062 -0.005232 0.968979 -0.033431 -0.001028 -0.001818 
dt_day -0.000554 0.000708 0.060967 0.001171 0.001031 0.000714 
dt_dayofy -0.001124 -0.005282 0.970475 -0.032651 -0.000929 -0.001745 
dt_hour 1.000000 0.105231 -0.001150 -0.016012 -0.015445 -0.017057 
dt_min 0.105231 1.000000 -0.005041 -0.075062 -0.197825 -0.161784 
dt_week_no -0.001150 -0.005041 1.000000 -0.028198 -0.001029 -0.001712 
dt_int -0.016012 -0.075062 -0.028198 1.000000 0.027027 0.022670 
ca_idx -0.015445 -0.197825 -0.001029 0.027027 1.000000 0.711480 
unit_idx -0.017057 -0.161784 -0.001712 0.022670 0.711480 1.000000 
scp_idx 0.018159 0.230438 -0.000521 0.016230 -0.302035 -0.417620 
station_idx -0.016686 -0.108720 -0.026729 0.223788 0.450012 0.668605 
linename_idx -0.007028 -0.101023 -0.003764 0.054294 -0.056877 -0.117951 
division_idx 0.012013 0.291330 0.002228 0.010168 -0.087393 -0.011747 
desc_idx 0.091890 0.360915 -0.002069 -0.216042 -0.037397 -0.012350 

 scp_idx station_idx linename_idx division_idx desc_idx 
entries -0.127529 -0.001799 0.076172 -0.162791 -0.018528 
exits -0.128658 -0.019425 0.084422 -0.128089 -0.012205 
dt_year 0.016085 0.224509 0.054080 0.009763 -0.213068 
dt_month -0.000523 -0.027903 -0.003617 0.002106 -0.002147 
dt_day -0.000172 0.000933 -0.001351 -0.000357 0.007103 
dt_dayofy -0.000546 -0.027375 -0.003726 0.002045 -0.001850 
dt_hour 0.018159 -0.016686 -0.007028 0.012013 0.091890 
dt_min 0.230438 -0.108720 -0.101023 0.291330 0.360915 
dt_week_no -0.000521 -0.026729 -0.003764 0.002228 -0.002069 
dt_int 0.016230 0.223788 0.054294 0.010168 -0.216042 
ca_idx -0.302035 0.450012 -0.056877 -0.087393 -0.037397 
unit_idx -0.417620 0.668605 -0.117951 -0.011747 -0.012350 
scp_idx 1.000000 -0.273493 0.016367 0.151899 -0.024920 
station_idx -0.273493 1.000000 -0.023277 0.043004 -0.043610 
linename_idx 0.016367 -0.023277 1.000000 0.035287 0.001989 
division_idx 0.151899 0.043004 0.035287 1.000000 -0.004743 
desc_idx -0.024920 -0.043610 0.001989 -0.004743 1.000000

In [13]:
df_desafio_num.columns

Out[ 40 ]: 
['entries',
 'exits',
 'dt_year',
 'dt_month',
 'dt_day',
 'dt_dayofy',
 'dt_hour',
 'dt_min',
 'dt_week_no',
 'dt_int',
 'ca_idx',
 'unit_idx',
 'scp_idx',
 'station_idx',
 'linename_idx',
 'division_idx',
 'desc_idx']

In [14]:
# Verorização de features
colunas = ['exits', 'dt_year', 'dt_month', 'dt_day', 'dt_dayofy', 'dt_hour', 'dt_min', 'dt_week_no', 'dt_int', 'ca_idx', 'unit_idx', 'scp_idx', 'station_idx', 'linename_idx', 'division_idx', 'desc_idx']
vectorAssembler = VectorAssembler(inputCols = colunas, outputCol = 'features')
vdf_desafio_num = vectorAssembler.transform(df_desafio_num)
vdf_desafio_num.take(1)

# Seleciona features de interesse
vdf_desafio_num = vdf_desafio_num.select(['features', 'entries'])
vdf_desafio_num.show(3)

# Split para dados de treino
splits = vdf_desafio_num.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

# Linear model
lr = LinearRegression(featuresCol = 'features', labelCol='entries', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

+--------------------+-------+
 features|entries|
+--------------------+-------+
[2380798.0,2010.0...|2827995|
[834908.0,2010.0,...| 704052|
[2562997.0,2010.0...|2130785|
+--------------------+-------+
only showing top 3 rows

Coefficients: [0.666741597065,33825.6384437,1591.02893593,176.85786226,53.4731542645,289.825225738,-10074.3676436,377.738862367,0.00107513007628,-765.427479083,2332.56084501,-2623.79363443,-1005.33000969,3806.6317179,-288032.335406,11362.4652628]
Intercept: -67814071.31129354

In [15]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 2795376.013776
r2: 0.398383

In [16]:
train_df.describe().show()

+-------+------------------+
summary| entries|
+-------+------------------+
 count| 53142403|
 mean| 3220367.110011736|
 stddev|3604406.1092531136|
 min| 0|
 max| 19999960|
+-------+------------------+

In [17]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","entries","features").show(5)
lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="entries",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+-------+--------------------+
 prediction|entries| features|
+------------------+-------+--------------------+
1719758.1391474605| 425585|(16,[0,1,2,3,4,7,...|
1748048.7661632895| 510913|(16,[0,1,2,3,4,7,...|
1764925.6301308423| 576853|(16,[0,1,2,3,4,7,...|
1811810.8254876733| 809032|(16,[0,1,2,3,4,7,...|
 1845967.333626926| 929607|(16,[0,1,2,3,4,7,...|
+------------------+-------+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.398122

#### Exploratory data analysis (EDA) - Part 3

In [19]:
#df_graficos = df_desafio_v2.filter(col('dt_year')=='2017')
df_graficos = df_desafio_v2.groupBy(
  'dt_month_year',
  'dt_year',
  'dt_month',
  'dt_day',
  'dt_hour'
).agg(
  sum('entries'),
  sum('exits')
).orderBy(
  "dt_month_year"
).toPandas()

#df_desafio_num.describe().toPandas().transpose()
#df_desafio_num.dtypes


In [20]:
#display(df_graficos)